In [1]:
! pip install -r ../requirements.txt


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import numpy as np
import torch
import copy

import sys
import os
sys.path.append(os.path.expanduser("~/repos/NeurOps/pytorch"))
from neurops import *

from growprune import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
hidden = [16, 16]
regression = False

base_data = ToyData(64, 32, 5000, regression=regression)
train_loader, val_loader, test_loader = split_dataset(base_data, val_size = 0.1, test_size = 0.1, batch_size = 128)



model = ModSequential(
        ModLinear(64, hidden[0], masked=True, prebatchnorm=True, learnable_mask=True),
        ModLinear(hidden[0], hidden[1], masked=True, prebatchnorm=True, learnable_mask=True),
        ModLinear(hidden[1], 1 if regression else 2, masked=True, prebatchnorm=True),
        track_activations=True,
        track_auxiliary_gradients=True
    ).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss() if regression else torch.nn.CrossEntropyLoss()

print("This model has {} effective parameters.".format(model.parameter_count(masked = True)))
#print("The conversion factor of this model is {} after layer {}.".format(model.conversion_factor, model.conversion_layer))

This model has 1378 effective parameters.


In [10]:
train(model, train_loader, optimizer, criterion, epochs=25, val_loader=val_loader, device=device, regression=regression, verbose=False)

Validation: Average loss: 0.0062, Accuracy: 215/450 (47.78%)
Validation: Average loss: 0.0061, Accuracy: 234/450 (52.00%)
Validation: Average loss: 0.0060, Accuracy: 262/450 (58.22%)
Validation: Average loss: 0.0059, Accuracy: 273/450 (60.67%)
Validation: Average loss: 0.0058, Accuracy: 291/450 (64.67%)
Validation: Average loss: 0.0056, Accuracy: 313/450 (69.56%)
Validation: Average loss: 0.0053, Accuracy: 327/450 (72.67%)
Validation: Average loss: 0.0050, Accuracy: 331/450 (73.56%)
Validation: Average loss: 0.0046, Accuracy: 348/450 (77.33%)
Validation: Average loss: 0.0041, Accuracy: 356/450 (79.11%)
Validation: Average loss: 0.0037, Accuracy: 370/450 (82.22%)
Validation: Average loss: 0.0031, Accuracy: 381/450 (84.67%)
Validation: Average loss: 0.0027, Accuracy: 399/450 (88.67%)
Validation: Average loss: 0.0023, Accuracy: 412/450 (91.56%)
Validation: Average loss: 0.0020, Accuracy: 417/450 (92.67%)
Validation: Average loss: 0.0018, Accuracy: 418/450 (92.89%)
Validation: Average loss

In [22]:
modded_data = copy.deepcopy(base_data)
modded_data.shift_distribution(-1, 100)
modded_train_loader, modded_val_loader, modded_test_loader = split_dataset(modded_data, val_size = 0.1, test_size = 0.1, batch_size = 128)

In [23]:
train(model, modded_train_loader, optimizer, criterion, epochs=25, val_loader=modded_val_loader, device=device, regression=regression, verbose=False)
test(model, val_loader, criterion, device=device, regression=regression)

Validation: Average loss: 0.0000, Accuracy: 449/450 (99.78%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validation: Average loss: 0.0000, Accuracy: 450/450 (100.00%)
Validatio